# Bert를 사용한 문장 간 관계 분류

2개의 문장을 주고, 3개의 카테고리 중의 하나로 분류한다.

연관, 중립, 상반.

<br>

문장 데이터를 WordPieceTokenizer로 토크나이징하고 

결과 토큰들을 vocab 파일로 저장.

이후 BertTokenizer가 이 파일을 읽어 처리.

<br>

pretrained된 bert layer가 아닌 새로 학습한다.



copy from https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/blob/master/7.PRETRAIN_METHOD/7.2.2.bert_finetune_KorNLI.ipynb

# 필요 라이브러리 설치

In [1]:
!pip install transformers==3.0.2
!pip install sentencepiece

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from transformers import BertTokenizer
from transformers import TFBertModel

import tensorflow as tf

In [28]:
#random seed 고정
tf.random.set_seed(49)
np.random.seed(49)

SEQ_LENGTH = 128
BERT_MODEL_NAME = 'bert-base-multilingual-cased'
CUSTOM_VOCAB_FILE = 'custom_vocab.txt'

# 데이터

## 데이터 다운로드

In [3]:
!git clone https://github.com/kakaobrain/KorNLUDatasets

fatal: destination path 'KorNLUDatasets' already exists and is not an empty directory.


In [4]:
!wc ./KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv

  550153  8595590 78486224 ./KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv


## 데이터 로딩

In [29]:
df = pd.read_csv("KorNLUDatasets/KorNLI/snli_1.0_train.ko.tsv", delimiter = '\t', quoting = 3)

In [30]:
df.head()

,sentence1,sentence2,gold_label
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment


## 카테고리 인덱스 만들기

In [31]:
df.gold_label = df.gold_label.astype('category')

In [32]:
df['category'] = df.gold_label.cat.codes

In [33]:
df.head()

,sentence1,sentence2,gold_label,category
0,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 경쟁을 위해 말을 훈련시키고 있다.,neutral,2
1,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,한 사람이 식당에서 오믈렛을 주문하고 있다.,contradiction,0
2,말을 탄 사람이 고장난 비행기 위로 뛰어오른다.,사람은 야외에서 말을 타고 있다.,entailment,1
3,카메라에 웃고 손을 흔드는 아이들,그들은 부모님을 보고 웃고 있다,neutral,2
4,카메라에 웃고 손을 흔드는 아이들,아이들이 있다,entailment,1


In [34]:
category_names = list(df.gold_label.cat.categories)
print(category_names)

['contradiction', 'entailment', 'neutral']


## 데이터 섞기

In [35]:
df = df.sample(frac=1).reset_index(drop=True) 

df.head()

,sentence1,sentence2,gold_label,category
0,남자들이 축구 경기를 하고 있다.,대화를 나누는 남자들,neutral,2
1,두 명의 자전거 타는 사람들이 말을 하면서 거리를 지나간다.,누가 더 빠른지 주장하는 두 명의 자전거 타는 사람.,neutral,2
2,한 남자와 한 여자가 눈을 크게 뜨고 무언가를 바라보며 인도에 멈춰 섰다.,남자와 여자가 넓은 눈으로 무언가를 응시하고 있다.,entailment,1
3,"한 무리의 아이들, 두 개의 분리된 팀, 녹색과 빨간색, 축구를 한다.",두 팀의 아이들이 플레이오프 경기를 하고 있다.,neutral,2
4,한 사려 깊은 여자가 대도시의 샛길을 걷고 있다.,한 여자가 길을 걷는다.,entailment,1


## 필요 입출력 값 준비

In [36]:
sentences1 = df.sentence1.values.copy().astype(np.str)
sentences2 = df.sentence2.values.copy().astype(np.str)
labels = df.category.values.copy().astype(np.int)

In [37]:
print(sentences1.shape)
print(sentences2.shape)
print(labels.shape)

(550152,)
(550152,)
(550152,)


## Vocab 파일 만들기

### 전체 문장을 파일로 저장

In [40]:
all_sentence = []
all_sentence.extend(sentences1)
all_sentence.extend(sentences2)

In [42]:
print(all_sentence[0])
print(len(all_sentence))

남자들이 축구 경기를 하고 있다.
1100304


In [43]:
with open("all_sentence.txt", 'w') as f:
  for item in all_sentence:
    f.write("%s\n" % item)

In [44]:
!wc all_sentence.txt

 1100304  8045435 72434521 all_sentence.txt


### Tokenizer 생성

refer from https://gist.github.com/lovit/259bc1d236d78a77f044d638d0df300c

In [111]:
from tokenizers import BertWordPieceTokenizer

bert_wordpiece_tokenizer = BertWordPieceTokenizer(
    # vocab_file=None,
    # clean_text=True,
    # handle_chinese_chars=True,
    # strip_accents=False, # Must be False if cased model
    # lowercase=False,
    # wordpieces_prefix="##"    
    )
bert_wordpiece_tokenizer.train(
    files = 'all_sentence.txt',
    vocab_size = 50000,
    # min_frequency = 1,
    # limit_alphabet = 1000,
    # initial_alphabet = [],
    # special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"],
    # show_progress = True,
    # wordpieces_prefix = "##",
)

### 토크나이징 실행

In [112]:
vocab = bert_wordpiece_tokenizer.get_vocab()
# vocab = {'일부는': 5627, '돌다': 8717, '유연한': 17619, '나이든': 844, ... }

vocab = sorted(vocab, key=lambda x: vocab[x])
# vocab = ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '!', '"', '#', ... ]

from unicodedata import normalize
# vocab = [..., '##ㅇㅜㅓ, 'ㅎㅐ', 'ㅌㅏㄱㅗ', 'ㄱㅕㅇ', 'ㅇㅑ', 'ㅇㅕㄴ', ... ]
vocab = [normalize('NFKC', i) for i in vocab]  # 초중종성 분리된걸 묶는다 
# vocab = [..., '##워', '해', '타고', '경', '야', '연', ... ]
print(vocab[:20])
print(vocab[400:420])

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '!', '"', '#', '%', '&', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1']
['##워', '해', '타고', '경', '야', '연', '##비', '세', '##ᅡ른', '##는다', '##안', '##레', '희', '##리고', '무리의', '밖', '개가', '##ᆯ을', '흰', '건']


### vocab 파일 저장

In [113]:
with open(CUSTOM_VOCAB_FILE, 'w') as f:
  for item in vocab:
    f.write("%s\n" % item)

In [114]:
!wc {CUSTOM_VOCAB_FILE}

 50000  50000 534610 custom_vocab.txt


In [115]:
!head {CUSTOM_VOCAB_FILE}

[PAD]
[UNK]
[CLS]
[SEP]
[MASK]
!
"
#
%
&


## 실 사용할 Tokenizer 생성

In [116]:
tokenizer = BertTokenizer(vocab_file=CUSTOM_VOCAB_FILE, do_lower_case=False, model_max_length=SEQ_LENGTH)

In [117]:
tokenized = tokenizer("남자들이 축구 경기를 하고 있다.", text_pair="대화를 나누는 남자들", max_length=20, padding='max_length')
print("original sentence  :", "남자들이 축구 경기를 하고 있다.", "대화를 나누는 남자들")
print("tokens             :", tokenizer.convert_ids_to_tokens(tokenized['input_ids']))
print("token id           :", tokenized['input_ids'])
print("attention mask     :", tokenized['attention_mask'])
print("token type         :", tokenized['token_type_ids'])

original sentence  : 남자들이 축구 경기를 하고 있다. 대화를 나누는 남자들
tokens             : ['[CLS]', '남자들이', '축구', '경기를', '하고', '있다', '.', '[SEP]', '대화를', '나누는', '남자들', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
token id           : [2, 685, 550, 1423, 328, 243, 16, 3, 1778, 4918, 1686, 3, 0, 0, 0, 0, 0, 0, 0, 0]
attention mask     : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
token type         : [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [118]:
for token in ['[CLS]', '남자들이', '축구', '경기를', '하고', '있다', '.', '[SEP]']:
  print(token, "\t\t: ", tokenizer.vocab[token])

[CLS] 		:  2
남자들이 		:  685
축구 		:  550
경기를 		:  1423
하고 		:  328
있다 		:  243
. 		:  16
[SEP] 		:  3


![bert_input_architecture](https://user-images.githubusercontent.com/1250095/50039788-8e4e8a00-007b-11e9-9747-8e29fbbea0b3.png)

## x, y 생성


tokernizer 사용 중에 경고 메시지가 많이 뜬다. 억제한다.


In [119]:
import logging
logging.basicConfig(level=logging.ERROR)

In [120]:
def build_model_input(sentences1, sentences2):
  input_ids = []
  attention_masks = []
  token_type_ids = []

  for sentence1, sentence2 in zip(sentences1, sentences2):
    tokenized = tokenizer(sentence1, text_pair=sentence2, max_length=SEQ_LENGTH, padding='max_length')
    # tokenized = {'input_ids': [101, ...], 'token_type_ids': [0, ...], 'attention_mask': [1, ...]}
    input_ids.append(tokenized['input_ids'][:SEQ_LENGTH]) # 버그인지 몰라도 SEQ_LENGTH이상이어도 더 크게 나온다.
    attention_masks.append(tokenized['attention_mask'][:SEQ_LENGTH])
    token_type_ids.append(tokenized['token_type_ids'][:SEQ_LENGTH])

  return (np.array(input_ids), np.array(attention_masks), np.array(token_type_ids))


In [121]:
T = 10000
x = build_model_input(sentences1[:T], sentences2[:T])
y = labels[:T]

x는 다음과 같이 구성됨
```
x = (  token_ids,  attention_masks,  token_types   )
       x[0],       x[1],             [2]
```

<br>

첫번 째 데이터는 
```
   ( token_ids[0],  attention_masks[0], token_types[0] )
 = ( x[0][0],       x[1][0],            x[2][0]  )
```


## train/test 분리

In [122]:
def split_bert_data(x, y, test_ratio):
  split_index = int(len(y)*(1-test_ratio))
  train_x = (x[0][:split_index], x[1][:split_index], x[2][:split_index])
  test_x  = (x[0][split_index:], x[1][split_index:], x[2][split_index:])
  train_y, test_y = y[:split_index], y[split_index:]

  return (train_x, train_y), (test_x, test_y)

(train_x, train_y), (test_x, test_y) = split_bert_data(x, y, test_ratio=0.2)

In [123]:
print(sentences1[0], sentences2[0])
print(tokenizer.decode(train_x[0][0][:30]))
print(train_x[0][0][:30])
print(train_x[1][0][:30])
print(train_x[2][0][:30])

남자들이 축구 경기를 하고 있다. 대화를 나누는 남자들
[CLS] 남자들이 축구 경기를 하고 있다. [SEP] 대화를 나누는 남자들 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
[   2  685  550 1423  328  243   16    3 1778 4918 1686    3    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0]
[1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


# 학습

## 모델 생성

In [124]:
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Dense, Dropout

class TFBertClassifier(tf.keras.Model):
  def __init__(self):
    super(TFBertClassifier, self).__init__()

    self.bert = TFBertModel.from_pretrained(BERT_MODEL_NAME, trainable=True)
    self.dropout = Dropout(self.bert.config.hidden_dropout_prob)
    self.classifier = Dense(3, kernel_initializer=TruncatedNormal(self.bert.config.initializer_range), 
                            name="classifier", activation="softmax")

  def call(self, inputs, attention_mask=None, token_type_ids=None, training=True):

    outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
    # outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
    pooled_output = outputs[1] 
    v = self.dropout(pooled_output, training=training)
    out = self.classifier(v)

    return out

model = TFBertClassifier()


참고로 Bert의 default 설정은 다음과 같다.

In [125]:
print(model.bert.config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}



In [126]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

optimizer = Adam(3e-5)
loss = SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])


## 학습 실행

In [127]:
history = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
12379/12379 [==============================] - 6197s 499ms/step - loss: 0.8007 - accuracy: 0.6428 - val_loss: 0.7197 - val_accuracy: 0.6989
Epoch 2/5
12379/12379 [==============================] - 6168s 498ms/step - loss: 0.6630 - accuracy: 0.7230 - val_loss: 0.6837 - val_accuracy: 0.7186
Epoch 3/5
12379/12379 [==============================] - 6173s 499ms/step - loss: 0.5709 - accuracy: 0.7700 - val_loss: 0.7010 - val_accuracy: 0.7288
Epoch 4/5
12379/12379 [==============================] - 6162s 498ms/step - loss: 0.4771 - accuracy: 0.8147 - val_loss: 0.6684 - val_accuracy: 0.7346
Epoch 5/5
12379/12379 [==============================] - 6169s 498ms/step - loss: 0.3813 - accuracy: 0.8560 - val_loss: 0.8119 - val_accuracy: 0.7281


In [128]:
loss, acc = model.evaluate(test_x, test_y, batch_size=32)
print("loss =", loss)
print("acc =", acc)

3439/3439 [==============================] - 558s 162ms/step - loss: 0.8115 - accuracy: 0.7287
loss = 0.8114949464797974
acc = 0.7286855578422546


## 분류 실행

In [129]:
def do_classify(sentence1, sentence2):
  model_input = build_model_input([sentence1], [sentence2])
  y_ = model.predict(model_input)
  predicted = np.argmax(y_, axis=-1)[0]
  print(sentence1, sentence2, "-->", category_names[predicted], ",score :",y_[0][predicted])

do_classify("나는 왜 그런지 잘 모르겠다.", "나는 그 이유에 관해 확신한다.")
do_classify("나는 왜 그런지 잘 모르겠다.", "나는 그가 왜 학교를 전학했는지 모르겠다.")
do_classify("나는 왜 그런지 잘 모르겠다.", "나는 왜 그런 일이 일어났는지 모르겠어.")

나는 왜 그런지 잘 모르겠다. 나는 그 이유에 관해 확신한다. --> neutral ,score : 0.60375
나는 왜 그런지 잘 모르겠다. 나는 그가 왜 학교를 전학했는지 모르겠다. --> neutral ,score : 0.98842746
나는 왜 그런지 잘 모르겠다. 나는 왜 그런 일이 일어났는지 모르겠어. --> entailment ,score : 0.5320901
